In [1]:
# !pip install yahooquery[premium]
# !pip uninstall selenium
# !pip install selenium==3.5.0
import yahooquery
import requests
import json
from yahooquery import Ticker
import time
import json
import requests
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta
from datetime import datetime

import selenium

from selenium import webdriver

In [2]:
def tradingView(
    mktCapMin = 5000000000,
    div_yield_recent = 2,
    StochK = 25,
    StochD = 25,
    macd_macd = 0,
    macd_signal = 0,
):
    filter = [
        {
            "left": "volume",
            "operation": "nempty"
        },
        {
            "left": "type",
            "operation": "in_range",
            "right": [
                "stock",
                "dr",
                "fund"
            ]
        },
        {
            "left": "subtype",
            "operation": "in_range",
            "right": [
                "common",
                "foreign-issuer",
                "",
                "etf",
                "etf,odd",
                "etf,otc",
                "etf,cfd"
            ]
        },
        {
            "left": "exchange",
            "operation": "in_range",
            "right": [
                "AMEX",
                "NASDAQ",
                "NYSE"
            ]
        },
        {
            "left": "market_cap_basic",
            "operation": "egreater",
            "right": mktCapMin
        },
        {
            "left": "is_primary",
            "operation": "equal",
            "right": True
        },
        {
            "left": "Stoch.K",
            "operation": "less",
            "right": StochK
        },
        {
            "left": "Stoch.D",
            "operation": "less",
            "right": StochD
        },
        {
            "left": "MACD.macd",
            "operation": "less",
            "right": macd_macd
        },
        {
            "left": "MACD.signal",
            "operation": "less",
            "right": macd_signal
        },
        {
            "left": "dividend_yield_recent",
            "operation": "egreater",
            "right": div_yield_recent
        },
        {
            "left": "Stoch.RSI.K",
            "operation": "less",
            "right": 25
        },
        {
            "left": "Stoch.RSI.D",
            "operation": "less",
            "right": 25
        }
    ]
    options = {
        "lang": "en"
    }
    markets = {
        "america"
    }
    symbols = {
        "query": {
            "types": []
        },
        "tickers": []
    }
    columns = [
        #"logoid",
        "name",
        "description",
        "close",
        "change",
        "change_abs",
        "Recommend.All",
        "market_cap_basic",
        "price_earnings_ttm",
        "earnings_per_share_basic_ttm",
        "sector",
        "earnings_release_date",
        "earnings_release_next_date",
        "dividend_yield_recent"
    ]
    sort = {
        "sortBy": "volume",
        "sortOrder": "desc"
    }
    range = [
        0,
        150
    ]

    post_message = {}
    post_message['filter'] = filter
    post_message['options'] = options
    post_message['symbols'] = symbols
    post_message['columns'] = columns
    post_message['sort'] = sort
    post_message['range'] = range

    payload = json.dumps(post_message)


    url = "https://scanner.tradingview.com/america/scan"
    #     payload = self.payload
    headers = {
        'authority': 'scanner.tradingview.com',
        'accept': 'text/plain, */*; q=0.01',
        'accept-encoding': 'gzip, deflate, br',
        'accept-language': 'en-US,en;q=0.9',
        'content-type': 'application/x-www-form-urlencoded; charset=UTF-8',
        'dnt': '1',
        'origin': 'https//www.tradingview.com',
        'referer': 'https//www.tradingview.com/',
        'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="96", "Google Chrome";v="96"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'same-site',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36'
    }

    response = requests.request("POST", url, headers=headers, data=payload)

    response_json = response.json()['data']
    if not response_json:
        return []
    df0 = pd.DataFrame.from_dict(response_json)
    df0.drop(columns=['s'], inplace=True)
    df = pd.DataFrame(df0["d"].to_list(), columns=columns)
#     display(df)

    df['change'] = df['change'].round(2)
    df['dividend_yield_recent'] = df['dividend_yield_recent'].round(2)
    df['price_earnings_ttm'] = df['price_earnings_ttm'].round(2)
    df['earnings_per_share_basic_ttm'] = df['earnings_per_share_basic_ttm'].round(2)
#     display(df)
    myListOfTickers = df['name'].to_list()
#     executionTime = (time.time() - startTime)
#     tradingViewTime = tradingViewTime + executionTime

    # return render(request, 'options/options.html', results)
    return myListOfTickers



In [3]:
tvlist = tradingView()
display(tvlist)

yf_username = 'qcappartners@yahoo.com'
yf_password = 'Mcci2022!!'
# 'DilutedEPS', 'BasicEPS'
fin_types = ['DilutedEPS', 'BasicEPS']
types = ['TotalDebt', 'TotalAssets', 'EBIT', 'EBITDA', 'PeRatio']
financial_data = Ticker(tvlist, username=yf_username, password=yf_password).p_get_financial_data(types=types, trailing=False)
display(financial_data)

['VFC',
 'SIRI',
 'CVX',
 'AMCR',
 'D',
 'LUV',
 'TSN',
 'EMR',
 'WRK',
 'GEN',
 'MCD',
 'WTRG',
 'HAS',
 'CMS',
 'APD',
 'CWEN',
 'HESM',
 'IDA']

,asOfDate,periodType,currencyCode,EBIT,TotalAssets,TotalDebt
symbol,,,,,,
AMCR,1989-06-30,12M,USD,2.478958e+08,2.267419e+09,7.737780e+08
AMCR,1990-06-30,12M,USD,3.510129e+08,2.994989e+09,9.286344e+08
AMCR,1991-06-30,12M,USD,3.623682e+08,2.828586e+09,5.752030e+08
AMCR,1992-06-30,12M,USD,3.812757e+08,3.250620e+09,7.946164e+08
AMCR,1993-06-30,12M,USD,3.524579e+08,3.344093e+09,8.241034e+08
...,...,...,...,...,...,...
WTRG,2017-12-31,12M,USD,3.449930e+08,6.332463e+09,2.146801e+09
WTRG,2018-12-31,12M,USD,2.772210e+08,6.964496e+09,2.567408e+09
WTRG,2019-12-31,12M,USD,3.369090e+08,9.361985e+09,3.096691e+09


In [4]:
# display(financial_data)
print(len(tvlist))
data = financial_data
data = data.reset_index()
print(len(data['symbol'].unique()))

18
18
